In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from lifelines.utils import concordance_index

In [3]:
npz_data = np.load('fusion_features_origin.npz')
features = npz_data['features']  # Shape (144, 1024)
patient_ids = npz_data['patient_ids']  # Shape (144,)

event_data = pd.read_csv('processed_data.csv')

In [4]:
feature_columns = [f'feature_{i}' for i in range(features.shape[1])]
features_df = pd.DataFrame(features, columns=feature_columns)

features_df['Case ID'] = patient_ids
features_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_1015,feature_1016,feature_1017,feature_1018,feature_1019,feature_1020,feature_1021,feature_1022,feature_1023,Case ID
0,0.0,0.0,0.0,0.389201,0.0,0.0,0.0,1.410983,0.000000,0.066649,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.002345,0.000000,0.0,lung_001
1,0.0,0.0,0.0,0.247796,0.0,0.0,0.0,1.444206,0.432007,0.359523,...,0.000000,0.0,0.082699,0.0,0.084822,0.000000,0.000000,0.000000,0.0,lung_002
2,0.0,0.0,0.0,0.296023,0.0,0.0,0.0,1.438157,0.000000,0.000000,...,0.471473,0.0,0.218094,0.0,0.000000,0.000000,0.000000,1.525748,0.0,lung_003
3,0.0,0.0,0.0,0.504906,0.0,0.0,0.0,1.296884,0.848339,0.000000,...,0.596818,0.0,0.048936,0.0,0.044266,0.000000,0.084124,1.500630,0.0,lung_004
4,0.0,0.0,0.0,0.285188,0.0,0.0,0.0,1.161725,0.425602,0.006042,...,0.507732,0.0,0.000000,0.0,0.000000,0.007567,0.000000,0.000000,0.0,lung_005


In [5]:
event_data.head()

,Case ID,Age,Weight (lbs),Gender,Ethnicity,Smoking status,%GG,Tumor Location (choice=RUL),Tumor Location (choice=RML),Tumor Location (choice=RLL),...,Std,Min,Max,Median,SurfaceArea,Elongation,Flatness,Roundness,Time to Event,Event
0,lung_001,79,146.0,0,2,1,0,0,1,1,...,194.164635,-812,154,-56.236328,826.137989,1.436361,1.446431,0.699813,3078,0
1,lung_002,65,195.0,0,1,2,0,0,1,1,...,173.439744,-829,144,-36.021484,1037.374063,1.367921,1.112439,0.781205,70,0
2,lung_003,65,173.5,1,2,0,0,1,1,1,...,172.357348,-815,290,23.177734,755.268235,1.387373,1.089084,0.822009,666,0
3,lung_004,67,173.5,1,2,1,0,1,1,1,...,254.147443,-1024,366,21.595703,912.514223,1.593605,1.631120,0.639694,1172,0
4,lung_005,84,145.0,1,4,1,0,1,0,1,...,107.583454,-783,391,7.496094,2432.305090,1.278080,1.166290,0.735654,1456,1


In [6]:
event_data_subset = event_data[['Case ID', 'Time to Event', 'Event']]
combined_df = pd.merge(features_df, event_data_subset, on='Case ID', how='inner')
combined_df.shape[0]

144

In [8]:
features_cols = [col for col in combined_df.columns if col.startswith('feature_')]
metadata_cols = [col for col in combined_df.columns if not col.startswith('feature_')]

X = combined_df[features_cols]
y_event = combined_df['Event']
metadata = combined_df[metadata_cols]

In [ ]:
k_features = 100

# Sử dụng SelectKBest với f_classif (ANOVA F-value) để chọn features tốt nhất
selector = SelectKBest(f_classif, k=k_features)
X_selected = selector.fit_transform(X, y_event)

# các features được chọn
selected_features_cols = np.array(features_cols)[selector.get_support()]
X_selected_df = pd.DataFrame(X_selected, columns=selected_features_cols)

c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [   0    4    5    6   24   26   29   39   42   43   47   49   50   51
   52   54   55   56   57   58   59   65   69   77   79   84   85   89
   91   94  102  107  119  121  123  124  125  130  133  137  139  141
  145  146  154  156  157  158  160  162  171  175  177  181  194  204
  206  207  210  211  218  231  235  237  238  240  242  243  245  251
  254  256  259  264  266  268  269  270  271  272  277  278  280  282
  283  285  292  296  298  307  308  309  312  323  326  327  328  329
  333  334  336  337  339  341  343  345  350  353  355  356  361  368
  370  372  379  387  388  395  397  399  402  406  409  418  424  425
  426  429  434  437  440  446  455  457  464  474  475  479  484  486
  490  492  496  497  501  502  503  508  516  518  520  523  529  532
  542  545  546  547  552  553  554  559  568  572  576  577  579  580
  581  583  587  

In [10]:
print(f"Số chiều ban đầu: {X.shape[1]}")
print(f"Số chiều sau feature selection: {X_selected_df.shape[1]}")

Số chiều ban đầu: 1024
Số chiều sau feature selection: 100


In [11]:
selected_df = pd.concat([X_selected_df, metadata], axis=1)

In [12]:
train_df, test_df = train_test_split(selected_df, test_size=0.2, random_state=42)

In [13]:
train_df = train_df.drop(columns=['Case ID'])
test_df = test_df.drop(columns=['Case ID'])

In [14]:
cph = CoxPHFitter(penalizer=0.1, l1_ratio=0.5)
cph.fit(train_df, duration_col='Time to Event', event_col='Event')

c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\lifelines\utils\__init__.py:1100: ConvergenceWarning: Column(s) ['feature_112', 'feature_515', 'feature_710'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


<lifelines.CoxPHFitter: fitted with 115 total observations, 73 right-censored observations>

In [15]:
cph.print_summary()  # access the individual results using cph.summary
with open("results_CoxPH_kFeat_origin.txt", "w") as f:
    f.write(cph.summary.to_string() + "\n")
    f.write(f"Concordance Index: {cph.concordance_index_}\n")

<lifelines.CoxPHFitter: fitted with 115 total observations, 73 right-censored observations>
             duration col = 'Time to Event'
                event col = 'Event'
                penalizer = 0.1
                 l1 ratio = 0.5
      baseline estimation = breslow
   number of observations = 115
number of events observed = 42
   partial log-likelihood = -165.27
         time fit was run = 2025-04-26 16:39:07 UTC

---
              coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                     
feature_10   -0.00      1.00      0.01           -0.01            0.01                0.99                1.01
feature_15    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_23   -0.61      0.55      4.73           -9.88            8.67                0.00             5828.09
feature_28   -0.20      0.81      0.78           -1.73            1.32                0.18                3.73
feature_31    0.00      1.00      0.00           -0.01            0.01                0.99                1.01
feature_40   -0.07      0.93      0.58           -1.21            1.06                0.30                2.89
feature_44   -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_48   -0.16      0.85      3.18           -6.39            6.06                0.00              429.88
feature_62   -1.50      0.22      2.79           -6.97            3.96                0.00               52.49
feature_71    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_73    0.76      2.13      1.94           -3.05            4.56                0.05               95.80
feature_76   -2.82      0.06      4.49          -11.62            5.98                0.00              396.92
feature_82    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_86    0.16      1.17      0.28           -0.38            0.70                0.68                2.02
feature_88   -0.72      0.48      1.25           -3.17            1.72                0.04                5.56
feature_90   -0.00      1.00      0.00           -0.01            0.01                0.99                1.01
feature_96    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_99    0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_100  -0.48      0.62      1.04           -2.51            1.55                0.08                4.72
feature_112  20.18  5.82e+08     40.03          -58.28           98.64                0.00            6.92e+42
feature_113  -0.07      0.93      0.45           -0.95            0.81                0.39                2.25
feature_118   0.00      1.00      0.00           -0.01            0.01                0.99                1.01
feature_120  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_126   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_128   0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_135  -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
feature_159   0.08      1.09      1.25           -2.37            2.54                0.09               12.63
feature_163   0.00      1.00      0.00           -0.01            0.01                0.99                1.01
feature_179   0.80      2.22      1.17           -1.50            3.09                0.22               22.08
feature_203   0.07      1.08      2.00           -3.85            4.00                0.02               54.34
feature_208   0.00  

In [16]:
test_ci = cph.score(test_df, scoring_method="concordance_index")
print(test_ci)
with open("results_CoxPH_kFeat_origin.txt", "a") as f:
    f.write(f"Concordance Index on Test: {test_ci}\n")

0.8054054054054054


In [17]:
k_values = [10, 20, 50, 100, 200, 300]
ci_scores = []

for k in k_values:
    # Chọn k features
    temp_selector = SelectKBest(f_classif, k=k)
    X_temp = temp_selector.fit_transform(X, y_event)
    
    # Tạo DataFrame
    temp_cols = np.array(features_cols)[temp_selector.get_support()]
    X_temp_df = pd.DataFrame(X_temp, columns=temp_cols)
    
    # Kết hợp với metadata
    temp_df = pd.concat([X_temp_df, metadata], axis=1)
    
    # Chia tập train/test
    temp_train, temp_test = train_test_split(temp_df, test_size=0.2, random_state=42)
    
    # Loại bỏ patient_id
    temp_train = temp_train.drop(columns=['Case ID'])
    temp_test = temp_test.drop(columns=['Case ID'])
    
    # Fit mô hình
    temp_cph = CoxPHFitter(penalizer=0.1, l1_ratio=0.5)
    temp_cph.fit(temp_train, duration_col='Time to Event', event_col='Event')
    
    # Đánh giá
    ci = temp_cph.score(temp_test, scoring_method="concordance_index")
    ci_scores.append(ci)

# Vẽ biểu đồ concordance index theo số lượng features
plt.figure(figsize=(10, 6))
plt.plot(k_values, ci_scores, marker='o')
plt.xlabel('Số lượng features')
plt.ylabel('Concordance Index')
plt.title('Concordance Index theo số lượng features')
plt.grid(True)
plt.savefig('concordance_by_features.png')
plt.close()

print("\nConcordance Index theo số lượng features:")
for k, ci in zip(k_values, ci_scores):
    print(f"k={k}: CI={ci:.4f}")

c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  76  104  150  289  307  318  319  354  394  434  440  442  463  536
  576  578  603  682  691  707  723  734  774  777  808  820  911  922
  932  939  947  960  985  998 1021] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\ADMIN\anaconda3\envs\DATN\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  76  104  150  289  307  318  319  354  394  434  440  442  463  536
  576  578  603  682  691  707  723  734  774  777  808  820  911  922
  932  939  947  960  985  998 1021] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\AD


Concordance Index theo số lượng features:
k=10: CI=0.7027
k=20: CI=0.7081
k=50: CI=0.7027
k=100: CI=0.7297
k=200: CI=0.7243
k=300: CI=0.7243
